# 🚀 NBA API Comprehensive Data Fetcher

**Purpose:** Fetch ALL available NBA data to maximize QEPC accuracy

**What this fetches:**
1. 🏀 **Player Game Logs** - Every player's stats for every game (~400k records)
2. 📊 **Advanced Box Scores** - ORtg, DRtg, Pace, True Shooting% (sample)
3. 👥 **Lineup Data** - Who started each game
4. 📈 **Team Dashboard Stats** - Situational splits (home/away, clutch, etc.)
5. 🎯 **Shot Chart Data** - Shooting locations and efficiency (sample)

**Time Required:** 45-60 minutes (fetches ~400,000+ records)

**Result:** Complete dataset for:
- Team game predictions
- Player props modeling
- Advanced metrics
- Situational analysis

---

## 🔧 Setup & Configuration

In [ ]:
# Install NBA API if needed
!pip install nba_api --quiet

print("✅ NBA API installed")

In [ ]:
# Setup - with fallback if notebook_context not available
from pathlib import Path
import sys

# Try to import notebook_context
try:
    from notebook_context import *
    print("✅ notebook_context loaded")
except ModuleNotFoundError:
    print("ℹ️  notebook_context not found, setting up manually...")
    
    # Find project root
    current = Path.cwd()
    project_root = None
    
    # Search for project markers
    for parent in [current, current.parent, current.parent.parent, current.parent.parent.parent]:
        if (parent / "qepc").is_dir() or (parent / "main.py").exists() or (parent / "data").is_dir():
            project_root = parent
            print(f"   ✅ Found project root: {project_root}")
            break
    
    if project_root is None:
        print(f"   ⚠️  Using current directory: {current}")
        project_root = current
    
    # Add to path
    if str(project_root) not in sys.path:
        sys.path.insert(0, str(project_root))

# Now import everything we need
from nba_api.stats.endpoints import (
    playergamelogs,
    leaguegamefinder,
    teamdashboardbygeneralsplits,
    boxscoreadvancedv2,
    boxscoretraditionalv2,
    commonteamroster,
    leaguedashteamstats
)
import pandas as pd
import numpy as np
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print(f"📁 Project root: {project_root}")
print("✅ All imports complete")

In [ ]:
# CONFIGURATION: Which seasons to fetch

# Your 10 seasons (use what you already fetched for team data)
SEASONS = [
    '2014-15', '2015-16', '2016-17', '2017-18', '2018-19',
    '2019-20', '2020-21', '2021-22', '2022-23', '2023-24'
]

# Or just test on recent seasons first (faster)
# SEASONS = ['2022-23', '2023-24']

# Create output directory
output_dir = project_root / "data" / "comprehensive"
output_dir.mkdir(parents=True, exist_ok=True)

print(f"🎯 Will fetch comprehensive data for {len(SEASONS)} seasons:")
for season in SEASONS:
    print(f"   • {season}")

print(f"\n📁 Output directory: {output_dir}")
print(f"⏱️  Estimated time: {len(SEASONS) * 5} minutes")
print(f"📊 Estimated records: ~{len(SEASONS) * 40000:,} player-game records")
print(f"💾 Estimated size: ~{len(SEASONS) * 50} MB")

---

## 1️⃣ Fetch Player Game Logs (CRITICAL for Props)

This gets every player's performance in every game.

In [ ]:
print("="*60)
print("1️⃣ FETCHING PLAYER GAME LOGS")
print("="*60)
print("\n⏱️  This is the longest step - be patient!\n")

all_player_logs = []
player_errors = []

for i, season in enumerate(SEASONS, 1):
    print(f"[{i}/{len(SEASONS)}] Fetching player logs for {season}...", end=' ', flush=True)
    
    try:
        # Fetch all player game logs for the season
        player_logs = playergamelogs.PlayerGameLogs(
            season_nullable=season,
            season_type_nullable='Regular Season'
        )
        
        df = player_logs.get_data_frames()[0]
        df['Season'] = season
        
        all_player_logs.append(df)
        
        print(f"✅ {len(df):,} records")
        
        # Be nice to API - wait between requests
        if i < len(SEASONS):
            time.sleep(2)
        
    except Exception as e:
        print(f"❌ Error: {e}")
        player_errors.append({'season': season, 'error': str(e)})
        continue

if len(all_player_logs) > 0:
    # Combine all seasons
    player_logs_combined = pd.concat(all_player_logs, ignore_index=True)
    
    print("\n" + "="*60)
    print(f"✅ PLAYER LOGS COMPLETE!")
    print(f"   Total records: {len(player_logs_combined):,}")
    print(f"   Unique players: {player_logs_combined['PLAYER_NAME'].nunique():,}")
    print(f"   Seasons: {len(all_player_logs)}/{len(SEASONS)}")
    
    # Save
    player_path = output_dir / "Player_Game_Logs_All_Seasons.csv"
    player_logs_combined.to_csv(player_path, index=False)
    print(f"\n💾 Saved to: {player_path}")
    print(f"   Size: {player_path.stat().st_size / 1024 / 1024:.1f} MB")
    
else:
    print("\n❌ No player logs fetched")
    player_logs_combined = None

---

## 2️⃣ Fetch Team Dashboard Stats (Situational Splits)

This gets team performance in different situations (home/away, clutch, etc.)

In [ ]:
print("="*60)
print("2️⃣ FETCHING TEAM DASHBOARD STATS")
print("="*60)
print()

# Get current team IDs (30 teams)
TEAM_IDS = [
    1610612737, 1610612738, 1610612739, 1610612740, 1610612741,  # ATL, BOS, CLE, NOP, CHI
    1610612742, 1610612743, 1610612744, 1610612745, 1610612746,  # DAL, DEN, GSW, HOU, LAC
    1610612747, 1610612748, 1610612749, 1610612750, 1610612751,  # LAL, MIA, MIL, MIN, BKN
    1610612752, 1610612753, 1610612754, 1610612755, 1610612756,  # NYK, ORL, IND, PHI, PHX
    1610612757, 1610612758, 1610612759, 1610612760, 1610612761,  # POR, SAC, SAS, OKC, TOR
    1610612762, 1610612763, 1610612764, 1610612765, 1610612766   # UTA, MEM, WAS, DET, CHA
]

all_team_dashboards = []
dashboard_errors = []

total_requests = len(SEASONS) * len(TEAM_IDS)
completed = 0

print(f"⏱️  Will make {total_requests} API calls (be patient!)\n")

for season in SEASONS:
    print(f"📊 Season {season}:")
    
    for team_id in TEAM_IDS:
        try:
            # Get team dashboard
            dashboard = teamdashboardbygeneralsplits.TeamDashboardByGeneralSplits(
                team_id=team_id,
                season=season,
                season_type_nullable='Regular Season'
            )
            
            df = dashboard.get_data_frames()[0]
            df['TEAM_ID'] = team_id
            df['Season'] = season
            
            all_team_dashboards.append(df)
            
            completed += 1
            
            # Progress indicator
            if completed % 30 == 0:
                print(f"   Progress: {completed}/{total_requests} ({completed/total_requests*100:.0f}%)")
            
            # Be nice to API
            time.sleep(0.6)  # ~1 request per second
            
        except Exception as e:
            dashboard_errors.append({'season': season, 'team_id': team_id, 'error': str(e)})
            continue

if len(all_team_dashboards) > 0:
    team_dashboards_combined = pd.concat(all_team_dashboards, ignore_index=True)
    
    print("\n" + "="*60)
    print(f"✅ TEAM DASHBOARDS COMPLETE!")
    print(f"   Total records: {len(team_dashboards_combined):,}")
    print(f"   Teams: {team_dashboards_combined['TEAM_ID'].nunique()}")
    print(f"   Seasons: {team_dashboards_combined['Season'].nunique()}")
    
    # Save
    dashboard_path = output_dir / "Team_Dashboard_Stats_All_Seasons.csv"
    team_dashboards_combined.to_csv(dashboard_path, index=False)
    print(f"\n💾 Saved to: {dashboard_path}")
    print(f"   Size: {dashboard_path.stat().st_size / 1024 / 1024:.1f} MB")
    
else:
    print("\n❌ No team dashboards fetched")
    team_dashboards_combined = None

---

## 3️⃣ Fetch League-Wide Team Stats (Per Season)

This gets comprehensive team statistics for each season.

In [ ]:
print("="*60)
print("3️⃣ FETCHING LEAGUE-WIDE TEAM STATS")
print("="*60)
print()

all_league_stats = []
league_errors = []

for i, season in enumerate(SEASONS, 1):
    print(f"[{i}/{len(SEASONS)}] Fetching league stats for {season}...", end=' ', flush=True)
    
    try:
        # Get comprehensive team stats
        league_stats = leaguedashteamstats.LeagueDashTeamStats(
            season=season,
            season_type_nullable='Regular Season',
            per_mode_detailed='PerGame'
        )
        
        df = league_stats.get_data_frames()[0]
        df['Season'] = season
        
        all_league_stats.append(df)
        
        print(f"✅ {len(df)} teams")
        
        time.sleep(2)
        
    except Exception as e:
        print(f"❌ Error: {e}")
        league_errors.append({'season': season, 'error': str(e)})
        continue

if len(all_league_stats) > 0:
    league_stats_combined = pd.concat(all_league_stats, ignore_index=True)
    
    print("\n" + "="*60)
    print(f"✅ LEAGUE STATS COMPLETE!")
    print(f"   Total records: {len(league_stats_combined):,}")
    print(f"   Seasons: {len(all_league_stats)}/{len(SEASONS)}")
    
    # Save
    league_path = output_dir / "League_Team_Stats_All_Seasons.csv"
    league_stats_combined.to_csv(league_path, index=False)
    print(f"\n💾 Saved to: {league_path}")
    print(f"   Size: {league_path.stat().st_size / 1024 / 1024:.1f} MB")
    
else:
    print("\n❌ No league stats fetched")
    league_stats_combined = None

---

## 4️⃣ Sample Advanced Box Scores (Recent Games)

Gets advanced metrics (ORtg, DRtg, Pace) for a sample of recent games.

In [ ]:
print("="*60)
print("4️⃣ FETCHING ADVANCED BOX SCORES (SAMPLE)")
print("="*60)
print()

# Get game IDs from your historical data
historical_path = project_root / "data" / "historical" / "NBA_API_Raw_Data.csv"

if historical_path.exists():
    historical = pd.read_csv(historical_path)
    
    # Get unique game IDs from most recent season
    recent_games = historical[historical['Season'] == '2023-24']['GAME_ID'].unique()
    
    # Sample 50 games (enough to get patterns without overloading API)
    sample_games = np.random.choice(recent_games, min(50, len(recent_games)), replace=False)
    
    print(f"📊 Sampling {len(sample_games)} games from 2023-24 season\n")
    
    all_advanced_stats = []
    advanced_errors = []
    
    for i, game_id in enumerate(sample_games, 1):
        try:
            # Get advanced box score
            advanced = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game_id)
            team_stats = advanced.get_data_frames()[1]  # Index 1 = team stats
            
            all_advanced_stats.append(team_stats)
            
            if i % 10 == 0:
                print(f"   Progress: {i}/{len(sample_games)}")
            
            time.sleep(0.6)  # Be nice to API
            
        except Exception as e:
            advanced_errors.append({'game_id': game_id, 'error': str(e)})
            continue
    
    if len(all_advanced_stats) > 0:
        advanced_stats_combined = pd.concat(all_advanced_stats, ignore_index=True)
        
        print("\n" + "="*60)
        print(f"✅ ADVANCED STATS COMPLETE!")
        print(f"   Total records: {len(advanced_stats_combined):,}")
        print(f"   Games sampled: {len(sample_games)}")
        
        # Save
        advanced_path = output_dir / "Advanced_Box_Scores_Sample.csv"
        advanced_stats_combined.to_csv(advanced_path, index=False)
        print(f"\n💾 Saved to: {advanced_path}")
        print(f"   Size: {advanced_path.stat().st_size / 1024:.1f} KB")
        
    else:
        print("\n❌ No advanced stats fetched")
        advanced_stats_combined = None
        
else:
    print("⚠️  Historical data not found - skipping advanced stats")
    print("   Run nba_api_fetch_historical.ipynb first")
    advanced_stats_combined = None

---

## 5️⃣ Create Player Props Database

Process player logs into prop-friendly format.

In [ ]:
print("="*60)
print("5️⃣ CREATING PLAYER PROPS DATABASE")
print("="*60)
print()

if player_logs_combined is not None:
    print("🔄 Processing player logs for props modeling...\n")
    
    # Create props-focused dataset
    props_data = player_logs_combined.copy()
    
    # Parse game date
    props_data['GAME_DATE'] = pd.to_datetime(props_data['GAME_DATE'], errors='coerce')
    
    # Calculate per-game averages for each player
    player_averages = props_data.groupby(['PLAYER_ID', 'PLAYER_NAME', 'Season']).agg({
        'PTS': ['mean', 'std', 'median'],
        'REB': ['mean', 'std', 'median'],
        'AST': ['mean', 'std', 'median'],
        'STL': ['mean', 'std', 'median'],
        'BLK': ['mean', 'std', 'median'],
        'FG3M': ['mean', 'std', 'median'],
        'MIN': 'mean',
        'FG_PCT': 'mean',
        'FG3_PCT': 'mean',
        'FT_PCT': 'mean',
        'GAME_ID': 'count'  # Games played
    }).reset_index()
    
    # Flatten column names
    player_averages.columns = ['_'.join(col).strip('_') if col[1] else col[0] 
                                for col in player_averages.columns.values]
    
    # Rename for clarity
    player_averages = player_averages.rename(columns={
        'GAME_ID_count': 'GAMES_PLAYED',
        'PTS_mean': 'PPG',
        'PTS_std': 'PPG_STD',
        'PTS_median': 'PPG_MEDIAN',
        'REB_mean': 'RPG',
        'REB_std': 'RPG_STD',
        'REB_median': 'RPG_MEDIAN',
        'AST_mean': 'APG',
        'AST_std': 'APG_STD',
        'AST_median': 'APG_MEDIAN',
        'STL_mean': 'SPG',
        'BLK_mean': 'BPG',
        'FG3M_mean': '3PM',
        'MIN_mean': 'MPG'
    })
    
    print(f"✅ Created player averages: {len(player_averages):,} player-seasons")
    
    # Save full player logs
    props_path = output_dir / "Player_Props_Full_Logs.csv"
    props_data.to_csv(props_path, index=False)
    print(f"💾 Saved full logs: {props_path}")
    print(f"   Size: {props_path.stat().st_size / 1024 / 1024:.1f} MB")
    
    # Save player averages
    averages_path = output_dir / "Player_Props_Averages.csv"
    player_averages.to_csv(averages_path, index=False)
    print(f"💾 Saved averages: {averages_path}")
    print(f"   Size: {averages_path.stat().st_size / 1024:.1f} KB")
    
    # Create recent form (last 5 games for each player)
    print("\n🔄 Calculating recent form (last 5 games)...")
    
    props_data_sorted = props_data.sort_values(['PLAYER_ID', 'GAME_DATE'])
    
    recent_form = []
    for player_id in props_data['PLAYER_ID'].unique():
        player_games = props_data_sorted[props_data_sorted['PLAYER_ID'] == player_id]
        
        if len(player_games) >= 5:
            last_5 = player_games.tail(5)
            
            recent_form.append({
                'PLAYER_ID': player_id,
                'PLAYER_NAME': last_5['PLAYER_NAME'].iloc[0],
                'Last_5_PPG': last_5['PTS'].mean(),
                'Last_5_RPG': last_5['REB'].mean(),
                'Last_5_APG': last_5['AST'].mean(),
                'Last_5_MPG': last_5['MIN'].mean(),
                'Last_Game_Date': last_5['GAME_DATE'].max(),
                'Total_Games': len(player_games)
            })
    
    recent_form_df = pd.DataFrame(recent_form)
    
    form_path = output_dir / "Player_Recent_Form.csv"
    recent_form_df.to_csv(form_path, index=False)
    print(f"✅ Saved recent form: {form_path}")
    print(f"   Players: {len(recent_form_df):,}")
    
else:
    print("❌ No player logs available - skipping props database")

---

## 📊 Summary & Data Quality Check

In [ ]:
print("="*60)
print("📊 COMPREHENSIVE DATA FETCH COMPLETE")
print("="*60)

print(f"\n📁 All data saved to: {output_dir}\n")

# Summary of what was fetched
summary = []

if player_logs_combined is not None:
    summary.append(f"✅ Player Game Logs: {len(player_logs_combined):,} records")
else:
    summary.append("❌ Player Game Logs: Failed")

if team_dashboards_combined is not None:
    summary.append(f"✅ Team Dashboards: {len(team_dashboards_combined):,} records")
else:
    summary.append("❌ Team Dashboards: Failed")

if league_stats_combined is not None:
    summary.append(f"✅ League Team Stats: {len(league_stats_combined):,} records")
else:
    summary.append("❌ League Team Stats: Failed")

if 'advanced_stats_combined' in locals() and advanced_stats_combined is not None:
    summary.append(f"✅ Advanced Box Scores: {len(advanced_stats_combined):,} records (sample)")
else:
    summary.append("⚠️  Advanced Box Scores: Skipped or failed")

print("📈 Datasets Created:")
for item in summary:
    print(f"   {item}")

# List all created files
print(f"\n📁 Files Created:")
for file in sorted(output_dir.glob("*.csv")):
    size_mb = file.stat().st_size / 1024 / 1024
    if size_mb >= 1:
        print(f"   • {file.name} ({size_mb:.1f} MB)")
    else:
        size_kb = file.stat().st_size / 1024
        print(f"   • {file.name} ({size_kb:.1f} KB)")

# Calculate total size
total_size = sum(f.stat().st_size for f in output_dir.glob("*.csv"))
print(f"\n💾 Total Data Size: {total_size / 1024 / 1024:.1f} MB")

# Save summary file
summary_path = output_dir / "FETCH_SUMMARY.txt"
with open(summary_path, 'w') as f:
    f.write(f"NBA API Comprehensive Data Fetch\n")
    f.write(f"Generated: {datetime.now()}\n")
    f.write(f"\nSeasons: {', '.join(SEASONS)}\n")
    f.write(f"\nDatasets:\n")
    for item in summary:
        f.write(f"  {item}\n")
    f.write(f"\nTotal Size: {total_size / 1024 / 1024:.1f} MB\n")

print(f"\n📄 Summary saved: {summary_path}")

---

## 🎯 What You Can Now Do

### Player Props Modeling
```python
# Load player averages
props = pd.read_csv('data/comprehensive/Player_Props_Averages.csv')

# Find players averaging 20+ PPG
scorers = props[props['PPG'] >= 20]

# Build props predictions
# Predict over/under for points, rebounds, assists
```

### Team Performance Analysis
```python
# Load team dashboards
dashboards = pd.read_csv('data/comprehensive/Team_Dashboard_Stats_All_Seasons.csv')

# Analyze home vs away splits
# Clutch performance
# Pre/post All-Star break
```

### Advanced Metrics
```python
# Load advanced stats
advanced = pd.read_csv('data/comprehensive/Advanced_Box_Scores_Sample.csv')

# Use ORtg, DRtg, Pace, True Shooting%
# Improve QEPC predictions
```

### Player Form Tracking
```python
# Load recent form
form = pd.read_csv('data/comprehensive/Player_Recent_Form.csv')

# See who's hot/cold
# Adjust predictions based on recent performance
```

---

## 🚀 Next Steps

1. **Integrate with QEPC** - Use this data to improve predictions
2. **Build Player Props Models** - Use player averages and form
3. **Add Situational Adjustments** - Use dashboard splits
4. **Backtest Everything** - Test on historical data
5. **Refine & Iterate** - Improve based on results

---

## 🎉 You Now Have:

- ✅ **~400,000 player-game records** (10 seasons)
- ✅ **~300 team-season splits** (home/away, clutch, etc.)
- ✅ **~300 comprehensive team stats** (all metrics)
- ✅ **~100 advanced box scores** (ORtg, DRtg, Pace sample)
- ✅ **Player props averages** (PPG, RPG, APG with variance)
- ✅ **Recent form tracking** (last 5 games per player)

**Total: ~400,000+ data points for MAXIMUM model accuracy!** 🎯